[Ciccone et al. (2017)](https://doi.org/10.1002/mus.25573) claims that the HRV metrics RMSSD and SD1 are identical. This paper is cited in the open-source toolbox Neurokit2 (in the function hrv_nonlinear). But when I use Neurokit2 to generate ECG data and compute RMSSD and SD1, they are not perfectly correlated:

In [1]:
import neurokit2 as nk
import numpy as np
import pandas as pd

In [2]:
default_hrv_df_list = []
sampling_rate = 250
peaks_list = []
# with 9 random seeds
for seed in range(9):
    # simulate 30 second ECG signal
    ecg = nk.ecg_simulate(duration=30, sampling_rate=sampling_rate, random_state=seed)
    # extract peaks
    peaks, info = nk.ecg_peaks(ecg, sampling_rate=sampling_rate)
    peaks_list.append(peaks)
    # extract HRV features using Neurokit2
    default_hrv_df_list.append(nk.hrv(peaks_list[seed]))

# concatenate 9 sets of extracted HRV features
default_hrv_df = pd.concat(default_hrv_df_list)

c:\programdata\miniconda3\lib\site-packages\neurokit2\signal\signal_psd.py:113: NeuroKitWarning: The duration of recording is too short to support a sufficiently long window for high frequency resolution. Consider using a longer recording or increasing the `min_frequency`
  category=NeuroKitWarning
c:\programdata\miniconda3\lib\site-packages\neurokit2\signal\signal_psd.py:113: NeuroKitWarning: The duration of recording is too short to support a sufficiently long window for high frequency resolution. Consider using a longer recording or increasing the `min_frequency`
  category=NeuroKitWarning
c:\programdata\miniconda3\lib\site-packages\neurokit2\signal\signal_psd.py:113: NeuroKitWarning: The duration of recording is too short to support a sufficiently long window for high frequency resolution. Consider using a longer recording or increasing the `min_frequency`
  category=NeuroKitWarning
c:\programdata\miniconda3\lib\site-packages\neurokit2\signal\signal_psd.py:113: NeuroKitWarning: The

In [3]:
default_hrv_df.corr(method="pearson").loc[["HRV_RMSSD", "HRV_SDSD", "HRV_SD1"],
                                           ["HRV_RMSSD", "HRV_SDSD", "HRV_SD1"]]

,HRV_RMSSD,HRV_SDSD,HRV_SD1
HRV_RMSSD,1.000000,0.999531,0.999531
HRV_SDSD,0.999531,1.000000,1.000000
HRV_SD1,0.999531,1.000000,1.000000


In [4]:
default_hrv_df.corr(method="spearman").loc[["HRV_RMSSD", "HRV_SDSD", "HRV_SD1"],
                                           ["HRV_RMSSD", "HRV_SDSD", "HRV_SD1"]]

,HRV_RMSSD,HRV_SDSD,HRV_SD1
HRV_RMSSD,1.000000,0.979088,0.979088
HRV_SDSD,0.979088,1.000000,1.000000
HRV_SD1,0.979088,1.000000,1.000000


Equation 1 in [Ciccone et al. (2017)](https://doi.org/10.1002/mus.25573):
$$
\mathrm{RMSSD}=\sqrt{\frac{\sum_{i=1}^{N-1}\left(R R_{i}-R R_{i+1}\right)^{2}}{N-1}}
$$

In [5]:
def ciccone2017_rmssd(rri):
    return np.sqrt(np.sum((rri[:-1] - rri[1:]) ** 2) / (len(rri) - 1))

Equation 2 in [Ciccone et al. (2017)](https://doi.org/10.1002/mus.25573):

$$
\mathrm{SD} 1=\sqrt{\frac{\sum_{i=1}^{N-1}\left(\frac{1}{\sqrt{2}} R R_{i}-\frac{1}{\sqrt{2}} R R_{i+1}\right)^{2}}{N-1}}
$$

In [6]:
def ciccone2017_sd1(rri):
    return np.sqrt(
        np.sum((((1 / np.sqrt(2)) * rri[:-1] - (1 / np.sqrt(2)) * rri[1:])) ** 2)
        / (len(rri) - 1)
    )

First equation in Section 2.1. in [Kim et al. (2012)](https://doi.org/10.1016/j.cmpb.2010.11.011), which was cited after Equation 2 in [Ciccone et al. (2017)](https://doi.org/10.1002/mus.25573):

$$
\mathrm{SD} 1=\sqrt{\operatorname{Var}\left(\frac{1}{\sqrt{2}} \mathrm{R} \mathrm{R}_{n}-\frac{1}{\sqrt{2}} \mathrm{R} \mathrm{R}_{n+1}\right)}
$$

In [7]:
def kim2012_sd1(rri):
    return np.sqrt(np.var((1 / np.sqrt(2)) * rri[:-1] - (1 / np.sqrt(2)) * rri[1:]))

In [8]:
from neurokit2.hrv.hrv_utils import _hrv_sanitize_input, _hrv_get_rri


def rmssd_sdsd_sd1(peaks, sampling_rate=1000, ddof=1, diff_method="Neurokit2", return_rri=False):
    # Sanitize input
    peaks = _hrv_sanitize_input(peaks)
    if isinstance(peaks, tuple):  # Detect actual sampling rate
        peaks, sampling_rate = peaks[0], peaks[1]
    # Compute R-R intervals (also referred to as NN) in milliseconds
    # rri, _ = _hrv_get_rri(peaks, sampling_rate=sampling_rate, interpolate=False)
    rri = _hrv_get_rri(peaks, sampling_rate=sampling_rate, interpolate=False)
    if return_rri:
        return rri
    if diff_method=="Neurokit2":
        diff_rri = np.diff(rri)
    else:
        diff_rri = rri[:-1] - rri[1:]
    nk_rmssd = np.sqrt(np.nanmean(diff_rri ** 2))
    nk_sdsd = np.nanstd(diff_rri, ddof=ddof)
    rri_plus = rri[1:]
    rri_n = rri[:-1]

    x1 = (rri_n - rri_plus) / np.sqrt(2)  # Eq.7
    nk_sd1 = np.std(x1, ddof=ddof)
    cc_rmssd = ciccone2017_rmssd(rri)
    cc_sd1 = ciccone2017_sd1(rri)
    km_sd1 = kim2012_sd1(rri)

    return pd.DataFrame(
        {
            "Neurokit2_RMSSD": [nk_rmssd],
            "Neurokit2_SDSD": [nk_sdsd],
            "Neurokit2_SD1": [nk_sd1],
            "Ciccone2017_RMSSD": [cc_rmssd],
            "Ciccone2017_SD1": [cc_sd1],
            "Kim2012_SD1": [km_sd1]
        }
    )

In [9]:
# diff_method_list = ["other", "Neurokit2"]
diff_method_list = ["Neurokit2"]
ddof_list = [0, 1]
corr_method_list = ["pearson", "spearman"]
for diff_method in diff_method_list:
    for ddof in ddof_list:
        ddof_hrv_df_list = []
        for peaks in peaks_list:
            ddof_hrv_df_list.append(rmssd_sdsd_sd1(peaks, sampling_rate=sampling_rate, ddof=ddof, diff_method=diff_method))
            ddof_hrv_df = pd.concat(ddof_hrv_df_list)
        print("\ndiff method = " + str(diff_method))
        print("\nddof = " + str(ddof))
        for corr_method in corr_method_list:
            print(corr_method)
            print(ddof_hrv_df.corr(method=corr_method).loc[["Neurokit2_SD1", "Kim2012_SD1"],
                                                           ["Neurokit2_SD1", "Kim2012_SD1"]])


diff method = Neurokit2

ddof = 0
pearson
               Neurokit2_SD1  Kim2012_SD1
Neurokit2_SD1            1.0          1.0
Kim2012_SD1              1.0          1.0
spearman
               Neurokit2_SD1  Kim2012_SD1
Neurokit2_SD1            1.0          1.0
Kim2012_SD1              1.0          1.0

diff method = Neurokit2

ddof = 1
pearson
               Neurokit2_SD1  Kim2012_SD1
Neurokit2_SD1       1.000000     0.999997
Kim2012_SD1         0.999997     1.000000
spearman
               Neurokit2_SD1  Kim2012_SD1
Neurokit2_SD1            1.0          1.0
Kim2012_SD1              1.0          1.0


The pearson correlation is less than 1 between Neurokit2_SD1 and Kim2021_SD1 because the Kim2021_SD1 function used numpy's default for the variance, the population variance, while Neurokit2 uses the sample standard deviation (ddof=1).

In [10]:
ddof_hrv_df.corr(method="pearson")

,Neurokit2_RMSSD,Neurokit2_SDSD,Neurokit2_SD1,Ciccone2017_RMSSD,Ciccone2017_SD1,Kim2012_SD1
Neurokit2_RMSSD,1.000000,0.999531,0.999531,1.000000,1.000000,0.999519
Neurokit2_SDSD,0.999531,1.000000,1.000000,0.999531,0.999531,0.999997
Neurokit2_SD1,0.999531,1.000000,1.000000,0.999531,0.999531,0.999997
Ciccone2017_RMSSD,1.000000,0.999531,0.999531,1.000000,1.000000,0.999519
Ciccone2017_SD1,1.000000,0.999531,0.999531,1.000000,1.000000,0.999519
Kim2012_SD1,0.999519,0.999997,0.999997,0.999519,0.999519,1.000000


In [11]:
ddof_hrv_df.corr(method="spearman")

,Neurokit2_RMSSD,Neurokit2_SDSD,Neurokit2_SD1,Ciccone2017_RMSSD,Ciccone2017_SD1,Kim2012_SD1
Neurokit2_RMSSD,1.000000,0.979088,0.979088,1.000000,0.995825,0.979088
Neurokit2_SDSD,0.979088,1.000000,1.000000,0.979088,0.966667,1.000000
Neurokit2_SD1,0.979088,1.000000,1.000000,0.979088,0.966667,1.000000
Ciccone2017_RMSSD,1.000000,0.979088,0.979088,1.000000,0.995825,0.979088
Ciccone2017_SD1,0.995825,0.966667,0.966667,0.995825,1.000000,0.966667
Kim2012_SD1,0.979088,1.000000,1.000000,0.979088,0.966667,1.000000


Like Neurokit2_RMSSD & Neurokit2_SD1, Ciccone2017_RMSSD is also not perfectly correlated with Kim2012_SD1, regardless of the correlation method.